# Flood Adaptation: Minimal Model

This python script outlines a basic Agent-Based Model (ABM) developed using the Mesa framework, designed as a simplified representation of household adaptation to flood events in a social network context. The model serves as a template for understanding the fundamental mechanics of flood adapdtation with a given social network.

This directory is structured as follows:
 - A Mesa Agent class `Households` is defined in `agents.py`
 - A Mesa Agent class `Government` is defined in `agents.py` (this agent currently does not perform any action)
 - A Mesa Model class `AdaptationModel` is defined in `model.py`
 - This notebook `demo.ipynb` provides a demonstration of the model and its visualization.

Each household agent in the model represents a household with attributes and behaviors that determine its response to flood events. The flood depth attribute assigned to each household is currently random for demonstration purposes. The decision by each household to adapt is also arbitrary. In a more comprehensive model, this could be replaced with more sophisticated decision-making algorithms.

The current implementation includes several simplifications and arbitrary choices, such as random flood depths and basic adaptation logic based on random thresholds. These aspects are designed for clarity and ease of understanding and should be replaced with realistic and sound choices.

You are encouraged to modify, expand, and customize the model. This might include integrating real-world data, implementing more complex decision-making processes for agents, or exploring different network topologies and their effects on social dynamics.

The visualisation provided is basic. Additional aspects can be added for visualization and more in-depth analysis.

_This notebook was tested on 2023-11-13 with Mesa version 2.1.4 on Python 3.12.0._

# Running the model and visualizing the model domain and social network over time

In [1]:
from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx

# Initialize the Adaptation Model with 50 household agents.
model = AdaptationModel(number_of_households=50, number_of_steps = 20, flood_map_choice="harvey", network="watts_strogatz") # flood_map_choice can be "harvey", "100yr", or "500yr"

# Calculate positions of nodes for the network plot.
# The spring_layout function positions nodes using a force-directed algorithm,
# which helps visualize the structure of the social network.
pos = nx.spring_layout(model.G)

# Define a function to plot agents on the network.
# This function takes a matplotlib axes object and the model as inputs.
def plot_network(ax, model):
    # Clear the current axes.
    ax.clear()
    # Determine the color of each node (agent) based on their adaptation status.
    colors = ['blue' if agent.is_adapted else 'red' for agent in model.schedule.agents]
    # Draw the network with node colors and labels.
    nx.draw(model.G, pos, node_color=colors, with_labels=True, ax=ax)
    # Set the title of the plot with the current step number.
    ax.set_title(f"Social Network State at Step {model.schedule.steps}", fontsize=12)

# Generate the initial plots at step 0.
# Plot the spatial distribution of agents. This is a function written in the model.py
model.plot_model_domain_with_agents()

# Plot the initial state of the social network.
fig, ax = plt.subplots(figsize=(7, 7))
plot_network(ax, model)
plt.show()

# Run the model for 20 steps and generate plots every 5 steps.
for step in range(model.number_of_steps):
    model.step()

    if step == model.flood_step:
        break

    # Every 5 steps, generate and display plots for both the spatial distribution and network.
    # Note the first step is step 0, so the plots will be generated at steps 4, 9, 14, and 19, which are the 5th, 10th, 15th, and 20th steps.
    if (step + 1) % 5 == 0:
        # Plot for the spatial map showing agent locations and adaptation status.
        plt.figure(figsize=(10, 6))
        model.plot_model_domain_with_agents()

        # Plot for the social network showing connections and adaptation statuses.
        fig, ax = plt.subplots(figsize=(7, 7))
        plot_network(ax, model)
        plt.show()

UnboundLocalError: cannot access local variable 'income' where it is not associated with a value

The KPI's are: 
FloodDamageEstimated/FloodDamageActual
CostOfAdaptation
Income/IncomeLabel

Eventually, we could reduce the following dataframe even more. However, it is nice to see the full picture of the agents in this matter. Something to discuss. 

In [2]:
agent_data = model.datacollector.get_agent_vars_dataframe()
agent_data

FloodDepthEstimated  FloodDamageEstimated  HouseSize  \
Step AgentID                                                         
0    0                   0.476734          56433.457127        138   
     1                   0.000000              0.000000         78   
     2                   0.188526          37975.177475        135   
     3                   0.798572         100781.805512        210   
     4                   0.358948          42167.913852        114   
...                           ...                   ...        ...   
15   45                  0.475576          54752.869737        134   
     46                 13.031796         229308.000000        291   
     47                  1.617119          72700.737111        126   
     48                  0.384169         105427.862141        278   
     49                  0.250573          74663.063655        233   

              FloodDepthActual  FloodDamageActual OptimalMeasure  IsAdapted  \
Step AgentID                                                                  
0    0                0.000000           0.000000           None      False   
     1                0.000000           0.000000           None      False   
     2                0.000000           0.000000           None      False   
     3                0.000000           0.000000           None      False   
     4                0.000000           0.000000           None      False   
...                        ...                ...            ...        ...   
15   45               0.529152       56720.909956     Sandbags_0       True   
     46              15.410365      229308.000000           None      False   
     47               0.895709       62459.068934     Sandbags_0       True   
     48               0.269258       91833.826328     Sandbags_0       True   
     49               0.213677       69556.837235     Sandbags_0       True   

              CostOfAdaptation   IncomeLabel  Income  Savings  \
Step AgentID                                                    
0    0                       0          Poor    4292     0.00   
     1                       0          Poor    4914     0.00   
     2                       0          Poor    4669     0.00   
     3                       0  Middle-Class   29556     0.00   
     4                       0          Poor    6762     0.00   
...                        ...           ...     ...      ...   
15   45                    670  Middle-Class   33082   984.10   
     46                      0  Middle-Class   40866     0.00   
     47                    630  Middle-Class   34572  1098.60   
     48                   1390  Middle-Class   32805   250.25   
     49                   1165          Rich  104174  4043.70   

              OwnFloodPerception  NetworkPerception  
Step AgentID                                         
0    0                         2                  0  
     1                         3                  0  
     2                         2                  0  
     3                         3                  0  
     4                         3                  0  
...                          ...                ...  
15   45                        2                  2  
     46                        2                  2  
     47                        2                  2  
     48                        2                  2  
     49                        2                  2  

[800 rows x 13 columns]

In [10]:
step15 = agent_data.loc[1, :][['FloodDamageEstimated', 'OptimalMeasure', 'HouseSize']]
step15

,FloodDamageEstimated,OptimalMeasure,HouseSize
AgentID,,,
0,56433.457127,Drains,138
1,0.000000,None,78
2,37975.177475,Sandbags,135
3,92452.630534,Sandbags_0,210
4,42167.913852,Sandbags,114
5,0.000000,None,228
6,0.000000,None,109
7,102513.381520,Drains,202
8,48425.634849,Drains,151


In [4]:
#step20 = agent_data.loc[19, :][['FloodDamageActual', 'IsAdapted']]
#step20

If you want to investigate a single agent, change the number behind key= and in the slice to the same number
This will give you the agent you want to investigate
For instance, look at the graph and the blue numbers, pick one out, and investigate how many steps it takes to save up for a measure
Or, to see how the perception changes (barely overtime, but that is the discreet nature of it)

In [5]:
result_df_xs = agent_data.xs(key=45, level='AgentID', axis=0, drop_level=False)

# Using .loc indexer
agent = agent_data.loc[(slice(None), 45), :]

agent

,,FloodDepthEstimated,FloodDamageEstimated,HouseSize,FloodDepthActual,FloodDamageActual,OptimalMeasure,IsAdapted,CostOfAdaptation,IncomeLabel,Income,Savings,OwnFloodPerception,NetworkPerception
Step,AgentID,,,,,,,,,,,,,
0,45,0.675576,61224.749079,134,0.000000,0.000000,None,False,0,Middle-Class,33082,0.0,1,0
1,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
2,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
3,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
4,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
5,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
6,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
7,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2
8,45,0.475576,54752.869737,134,0.000000,0.000000,Sandbags_0,True,670,Middle-Class,33082,984.1,2,2


In [ ]:
print('The model flooded at step', model.flood_step)

In [ ]:
model_data = model.datacollector.get_model_vars_dataframe()
model_data